In [9]:
import os
import numpy as np
import pandas as pd
from tqdm import tqdm

import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import f1_score, classification_report

import cv2 


In [ ]:
DATA_PATH = "./Data/Clothes_Dataset"

classes = os.listdir(DATA_PATH)
print("Classes found:", classes)

X = []
y = []

for label in classes:
    class_folder = os.path.join(DATA_PATH, label)
    for img_file in tqdm(os.listdir(class_folder), desc=f"Loading {label}"):
        img_path = os.path.join(class_folder, img_file)
        img = cv2.imread(img_path)         # BGR format
        if img is None:
            continue
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        
        # resize everything to same shape
        img = cv2.resize(img, (64, 64))
        
        X.append(img)
        y.append(label)

X = np.array(X)
y = np.array(y)

print("Shape of X:", X.shape)
print("Shape of y:", y.shape)


Classes found: ['Blazer', 'Celana_Panjang', 'Celana_Pendek', 'Gaun', 'Hoodie', 'Jaket', 'Jaket_Denim', 'Jaket_Olahraga', 'Jeans', 'Kaos', 'Kemeja', 'Mantel', 'Polo', 'Rok', 'Sweter']


Loading Sweter: 100%|██████████| 500/500 [00:07<00:00, 69.10it/s]

Shape of X: (7500, 64, 64, 3)
Shape of y: (7500,)


In [ ]:

n_samples = X.shape[0]
X_flat = X.reshape(n_samples, -1)

print("Original shape:", X.shape)
print("Flattened shape:", X_flat.shape)


Original shape: (7500, 64, 64, 3)
Flattened shape: (7500, 12288)


In [ ]:
X_train, X_test, y_train, y_test = train_test_split(
    X_flat, y, test_size=0.20, stratify=y, random_state=42
)

print("Train set:", X_train.shape)
print("Test set:", X_test.shape)


Train set: (6000, 12288)
Test set: (1500, 12288)


In [5]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

print("Features normalized.")


Features normalized.


In [6]:
log_reg = LogisticRegression(
    multi_class="multinomial",
    solver="saga",
    max_iter=200,
    n_jobs=-1
)

log_reg.fit(X_train_scaled, y_train)

print("Model training complete.")


C:\Users\ishat\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\LocalCache\local-packages\Python312\site-packages\sklearn\linear_model\_logistic.py:1247: FutureWarning: 'multi_class' was deprecated in version 1.5 and will be removed in 1.7. From then on, it will always use 'multinomial'. Leave it to its default value to avoid this warning.
  warnings.warn(


Model training complete.


C:\Users\ishat\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\LocalCache\local-packages\Python312\site-packages\sklearn\linear_model\_sag.py:348: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(


In [7]:
y_pred = log_reg.predict(X_test_scaled)

macro_f1 = f1_score(y_test, y_pred, average="macro")
print(f"Macro F1 Score: {macro_f1:.4f}")

print("\nClassification Report:\n")
print(classification_report(y_test, y_pred))


Macro F1 Score: 0.3096

Classification Report:

                precision    recall  f1-score   support

        Blazer       0.12      0.09      0.10       100
Celana_Panjang       0.32      0.35      0.33       100
 Celana_Pendek       0.33      0.41      0.37       100
          Gaun       0.24      0.22      0.23       100
        Hoodie       0.33      0.28      0.30       100
         Jaket       0.40      0.42      0.41       100
   Jaket_Denim       0.29      0.28      0.29       100
Jaket_Olahraga       0.24      0.22      0.23       100
         Jeans       0.57      0.52      0.54       100
          Kaos       0.39      0.39      0.39       100
        Kemeja       0.30      0.27      0.28       100
        Mantel       0.28      0.26      0.27       100
          Polo       0.22      0.25      0.23       100
           Rok       0.26      0.25      0.26       100
        Sweter       0.35      0.48      0.41       100

      accuracy                           0.31      150